---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [95]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [120]:
def date_sorter():
    import re

#     04/20/2009; 04/20/09; 4/20/09; 4/3/09
#     \d{1,2}[/-]\d{1,2}[/-]\d{2,4}

#     04/20/2009; 04/20/09; 4/20/09; 4/3/09
#     Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
#     (?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>[\d{4}|\d{2}]) # use this because anexpection '4.9/36/308
#     (?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4})
#     (?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{2})  
#     (?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2})

#     20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
#     (?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4})

#     Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
#     (?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4})


#     Feb 2009; Sep 2009; Oct 2010
#     (?P<month>[A-Z][a-z]{3,}),?\.? (?P<year>\d{4})
#     r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]? (?P<year>\d{4})'

#     6/2008; 12/2009
#     r'(?P<month>\d{1,2})[/](?P<year>\d{4})'

#     2009; 2010
#     r'?P<year>\d{4}'


    month_dict = {'Jan': 1, 'Feb': 2,'Mar': 3,'Apr': 4,'May': 5,'Jun': 6,
                  'Jul': 7, 'Aug': 8,'Sep': 9, 'Oct': 10,'Nov': 11,'Dec': 12,   
                  'Janaury': 1, 'February': 2,'March': 3,'April': 4,'June': 6,
                  'July': 7, 'August': 8, 'September': 9,'October': 10,'November': 11,'Decemeber': 12,             
                  'Sin': 1, 'Age': 8}

    # work on mm-dd-yyyy format
    df_ans0 = df.str.extractall(r'(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4})')
    remain_index1 = ~df.index.isin([index[0] for index in df_ans0.index])

    df_ans1 = df_ans0.append(df[remain_index1].str.extractall(r'(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2})'))
    remain_index2 = ~df.index.isin([index[0] for index in df_ans1.index])

    # work on dd mmm yyyy format
    df_ans2 = df_ans1.append(df[remain_index2].str.extractall(r'(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4})'))
    remain_index3 = ~df.index.isin([index[0] for index in df_ans2.index])

    # work on Mar 20th, 2009 format
    df_ans3 = df_ans2.append(df[remain_index3].str.extractall(r'(?P<month>[A-Za-z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4})'))
    remain_index4 = ~df.index.isin([index[0] for index in df_ans3.index])

    # work on 'Feb 2009' format:
    df_ans4 = df_ans3.append(df[remain_index4].str.extractall(r'(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4})'))
    remain_index5 = ~df.index.isin([index[0] for index in df_ans4.index])

    # work on '6/2008' format
    df_ans5 = df_ans4.append(df[remain_index5].str.extractall(r'(?P<month>\d{1,2})[/](?P<year>\d{4})'))
    remain_index6 = ~df.index.isin([index[0] for index in df_ans5.index])

    # work on '2009' format
    df_ans6 = df_ans5.append(df[remain_index6].str.extractall(r'(?P<year>\d{4})'))
    # drop the useless column (last column)
    df_ans = df_ans6.iloc[:, :-1]

    # work on year
    df_ans['year'] = df_ans['year'].apply(lambda x: '19'+ x if len(x)==2 else x)
    # df_ans['year'] = df_ans['year'].apply(lambda x: str(x))

    # work on month
    df_ans['month'] = df_ans['month'].apply(lambda x: x[:3] if type(x) is str else x)
    df_ans = df_ans.replace(month_dict)
    # df_ans['month'] = df_ans['month'].astype('float')
    # # df_ans['month'] = df_ans['month'].apply(lambda x: str(x))

    df_ans.fillna(1, inplace=True)

    df_ans['year'] = df_ans['year'].apply(lambda x: str(x))
    df_ans['month'] = df_ans['month'].apply(lambda x: str(x))
    df_ans['day'] = df_ans['day'].apply(lambda x: str(x))

    df_ans['date'] = df_ans['month'] + '/' + df_ans['day'] + '/' + df_ans['year']
    df_ans['date'] = pd.to_datetime(df_ans['date'])

    df_ans = df_ans.sort_values(by='date')
    return_rank = pd.Series(list(df_ans.index.labels[0]))
    
    return return_rank

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33      95
34      73
35     108
36     156
37     332
38     182
39      82
40     351
41     278
42     214
43     155
44     223
45     473
46      49
47     317
48      11
49     319
50      40
51     418
52     165
53     370
54     382
55       3
56      50
57     363
58     219
59     465
60     237
61      23
62     342
63     204
64     258
65     315
66      27
67      93
68      17
69     488
70     303
71     395
72     283
73     309
74     419
75     123
76      19
77     117
78     232
79      72
80     189
81     369
82     318
83     493
84     239
85     148
86     105
87     336
88       6
89     200
90      81